In [1]:
!pip install fancyimpute mrmr-selection skfeature-chappers

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.4 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=17a01ea8f709f944e97b4a3b633eab830083d323a211e996a55cf6f580f13fd2
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=59ac1b8fdea6476348a11bde435a16176e81a1ff9500c7800ac2adc3c9cb25a1
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge, BayesianRidge, LogisticRegression, LassoCV, LinearRegression
from sklearn.feature_selection import mutual_info_classif, chi2, RFE
from mlxtend.feature_selection import SequentialFeatureSelector
from mrmr import mrmr_classif
from fancyimpute import IterativeImputer


import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [3]:
# Open and load the file from Drive
directory = '/content/drive/MyDrive/Colab Notebooks/Project/'
file_path = directory + 'Book2.csv'
with open(file_path) as f:
    data = pd.read_csv(f)

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Columns: 188 entries, Samples to Val
dtypes: float64(185), int64(1), object(2)
memory usage: 138.2+ KB


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,15298,Prostate cancer,AA,84.85,2.26,1.635,27.75,19.85,34.75,1.69,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,17585,Prostate cancer,EA,87.40,3.18,1.330,23.70,14.20,20.10,2.76,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
2,17848,Case control,EA,157.00,7.33,3.250,43.10,43.10,92.30,3.51,...,402.0,58.9,196.0,141.0,454.0,218.0,341.0,86.1,151.00,302.0
3,17893,Case control,EA,59.30,1.86,1.270,18.40,11.30,13.80,1.16,...,328.0,30.9,124.0,93.0,365.0,131.0,215.0,94.1,101.00,391.0
4,17962,Case control,EA,57.80,1.78,0.781,18.00,12.10,13.20,1.48,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.80,115.0


In [4]:
df = data.copy()

# shuffling the df
shuffled_df = shuffle(df, random_state=42)

# Encoding catergorical data to numerical data
df['Race'] = df['Race'].map({'EA': 0, 'AA': 1})
df['Status'] = df['Status'].map({'Case control': 0, 'Prostate cancer': 1})

df.head()


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,15298,1,1,84.85,2.26,1.635,27.75,19.85,34.75,1.69,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,17585,1,0,87.40,3.18,1.330,23.70,14.20,20.10,2.76,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
2,17848,0,0,157.00,7.33,3.250,43.10,43.10,92.30,3.51,...,402.0,58.9,196.0,141.0,454.0,218.0,341.0,86.1,151.00,302.0
3,17893,0,0,59.30,1.86,1.270,18.40,11.30,13.80,1.16,...,328.0,30.9,124.0,93.0,365.0,131.0,215.0,94.1,101.00,391.0
4,17962,0,0,57.80,1.78,0.781,18.00,12.10,13.20,1.48,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.80,115.0


In [5]:
# Check for NaN values
nan_check = df.isna().sum()
print(nan_check[nan_check > 0])

# Check total number of NaN values in df
total_nan = df.isna().sum().sum()
print("Total number of NaN values in the DataFrame (Before):", total_nan)

df

PC aa C34:4    1
PC aa C36:0    1
PC aa C36:1    1
PC aa C36:2    1
PC aa C36:3    1
              ..
Ser            1
Thr            2
Trp            1
Tyr            1
Val            1
Length: 160, dtype: int64
Total number of NaN values in the DataFrame (Before): 170


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,15298,1,1,84.85,2.26,1.635,27.75,19.85,34.75,1.69,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,17585,1,0,87.40,3.18,1.330,23.70,14.20,20.10,2.76,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
2,17848,0,0,157.00,7.33,3.250,43.10,43.10,92.30,3.51,...,402.0,58.9,196.0,141.0,454.0,218.0,341.0,86.1,151.00,302.0
3,17893,0,0,59.30,1.86,1.270,18.40,11.30,13.80,1.16,...,328.0,30.9,124.0,93.0,365.0,131.0,215.0,94.1,101.00,391.0
4,17962,0,0,57.80,1.78,0.781,18.00,12.10,13.20,1.48,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.80,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,20103,0,1,137.00,3.00,2.390,47.20,23.50,41.50,2.44,...,207.0,22.9,99.1,68.5,200.0,98.0,111.0,60.4,63.10,213.0
90,20319,0,1,182.00,4.81,2.470,52.60,27.80,46.20,2.81,...,389.0,49.3,201.0,158.0,514.0,136.0,204.0,108.0,140.00,495.0
91,20323,0,1,150.00,3.46,2.260,55.00,29.70,61.00,3.81,...,312.0,29.5,143.0,117.0,330.0,94.6,155.0,87.4,75.20,348.0
92,20361,0,1,133.00,2.74,2.130,45.00,19.80,39.90,2.46,...,300.0,35.0,151.0,87.1,510.0,163.0,247.0,73.0,105.00,324.0


In [6]:
df = df.drop(columns=['Samples'])

#NaN Value Treatment

In [7]:
# Using SVR for NaN value treatment
mice_svr_imputer = IterativeImputer(estimator=SVR(C=1.0, epsilon=0.1), max_iter=10, random_state=42)
df_svr_imputed = mice_svr_imputer.fit_transform(df)

imputed_df = pd.DataFrame(df_svr_imputed, columns=df.columns)
imputed_df.tail(20)

,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,lysoPC a C20:4,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
74,0.0,1.0,54.3,0.827,0.828,18.30,12.00,43.7,1.20,7.64,...,171.000000,31.700000,75.800000,68.100000,230.000000,103.000000,141.000000,59.100000,72.200000,255.000000
75,1.0,0.0,159.0,4.300,3.930,54.50,39.50,51.4,4.19,11.20,...,268.000000,38.800000,126.000000,111.000000,357.000000,189.000000,194.000000,83.300000,91.700000,345.000000
76,1.0,0.0,136.0,3.750,2.300,42.30,27.70,60.0,4.50,10.50,...,258.000000,29.000000,96.700000,94.700000,455.000000,112.000000,191.000000,71.400000,91.100000,290.000000
77,1.0,1.0,118.0,2.930,1.740,30.40,13.50,20.7,2.22,7.49,...,302.000000,50.100000,112.000000,133.000000,288.000000,149.000000,162.000000,117.000000,142.000000,509.000000
78,1.0,1.0,151.0,4.080,2.100,40.20,24.70,48.4,3.04,12.00,...,283.000000,32.700000,96.400000,107.000000,272.000000,147.000000,146.000000,97.700000,140.000000,416.000000
79,1.0,1.0,99.4,2.520,2.920,29.30,17.10,20.4,1.80,6.73,...,345.000000,46.000000,100.000000,127.000000,276.000000,145.000000,193.000000,88.700000,138.000000,374.000000
80,0.0,1.0,106.0,2.280,1.950,36.60,23.10,44.7,2.98,11.10,...,213.000000,31.200000,82.700000,69.000000,225.000000,101.000000,140.000000,50.100000,59.500000,256.000000
81,0.0,1.0,83.6,2.660,1.200,22.60,15.00,24.6,1.99,6.25,...,281.000000,38.500000,84.700000,98.600000,297.000000,130.000000,186.000000,60.900000,122.000000,269.000000
82,1.0,1.0,86.6,1.950,1.920,29.50,15.60,24.3,1.98,15.00,...,539.000000,49.700000,264.000000,150.000000,326.000000,117.000000,186.000000,138.000000,144.000000,552.000000
83,1.0,1.0,215.0,5.010,2.780,74.30,31.80,43.9,5.49,22.30,...,421.000000,48.500000,209.000000,126.000000,293.000000,174.500000,227.500000,115.500000,141.000000,439.500000


In [8]:
# Check for NaN values
nan_check = imputed_df.isna().sum()
print(nan_check[nan_check > 0])

# Check total number of NaN values in df
total_nan = imputed_df.isna().sum().sum()
print("Total number of NaN values in the DataFrame (after):", total_nan)

# imputed_df.tail(10)

Series([], dtype: int64)
Total number of NaN values in the DataFrame (after): 0


# Filter rows where Status is "Prostate cancer" (1)

In [9]:
df_race = imputed_df[imputed_df['Status'] == 1]
df_race.info()
df_race.head()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 0 to 83
Columns: 187 entries, Status to Val
dtypes: float64(187)
memory usage: 64.6 KB


,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,lysoPC a C20:4,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,1.0,1.0,84.85,2.26,1.635,27.75,19.85,34.75,1.69,11.25,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,1.0,0.0,87.40,3.18,1.330,23.70,14.20,20.10,2.76,10.20,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
5,1.0,1.0,118.00,2.95,2.420,34.80,23.80,41.90,3.05,10.20,...,338.0,53.1,142.0,126.0,389.0,169.0,230.0,91.4,186.00,411.0
13,1.0,0.0,163.00,11.70,26.700,92.80,16.00,20.00,1.47,5.00,...,342.0,39.5,164.0,132.0,418.0,133.0,159.0,88.9,128.00,304.0
14,1.0,0.0,134.00,6.01,2.600,34.50,25.60,31.20,4.85,14.00,...,399.0,37.6,172.0,122.0,332.0,144.0,146.0,62.0,111.00,399.0


In [10]:
X = df_race.drop(columns=['Race', 'Status'])
y = df_race['Race']

# feature selection

mRMR

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from mrmr import mrmr_classif

results = []

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Loop over K values from 10 to 100
for K in range(10, 101):
    # Use mRMR to select top K features
    top_features = mrmr_classif(X=X, y=y, K=K)

    # # Split the data into training and testing sets
    # X_train, X_test, y_train, y_test = train_test_split(X[top_features], y, test_size=0.2, random_state=42)

    # Select only the top K features from the consistent train/test sets
    X_train_top = X_train[top_features]
    X_test_top = X_test[top_features]

    # Random Forest classifier
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train_top, y_train)

    # Predict on the test data and calculate accuracy
    y_pred_mrmr = rf.predict(X_test_top)
    accuracy_mrmr = accuracy_score(y_test, y_pred_mrmr)

    # Format the accuracy to 5 decimal places
    accuracy_mrmr = round(accuracy_mrmr, 5)

    # Get feature importances specifically for the mRMR selected features
    importances = rf.feature_importances_

    # Create a DataFrame for feature importances
    feature_importances = pd.Series(importances, index=top_features)

    # Sort by importance score
    sorted_importances = feature_importances.sort_values(ascending=False)

    # Create dictionaries for feature rank and importance
    feature_rank = {feature: rank + 1 for rank, feature in enumerate(sorted_importances.index)}
    feature_importance = {feature: importance for feature, importance in zip(sorted_importances.index, sorted_importances.values)}

    # Store results for this K
    results.append({
        'K': K,
        'Accuracy': accuracy_mrmr,
        'Feature_Rank': feature_rank,
        'Feature_Importance': feature_importance
    })

# Convert results to DataFrame for saving to Excel
final_results = pd.DataFrame(results)
# final_results

file_path_csv = directory + 'mrmr_results.csv'
final_results.to_csv(file_path_csv, index=False)
print(f"Results saved to {file_path_csv}")

100%|██████████| 100/100 [00:05<00:00, 18.30it/s]


Results saved to /content/drive/MyDrive/Colab Notebooks/Project/mrmr_results.csv


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from mrmr import mrmr_classif

# results = []

# # Loop over K values from 10 to 100
# for K in range(10, 101):
#     # Use mRMR to select top K features
#     top_features = mrmr_classif(X=X, y=y, K=K)

#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(X[top_features], y, test_size=0.2, random_state=42)

#     # Random Forest classifier
#     rf = RandomForestClassifier(n_estimators=100, random_state=42)
#     rf.fit(X_train, y_train)

#     # Predict on the test data and calculate accuracy
#     y_pred_mrmr = rf.predict(X_test)
#     accuracy_mrmr = accuracy_score(y_test, y_pred_mrmr)

#     # Format the accuracy to 5 decimal places
#     accuracy_mrmr = round(accuracy_mrmr, 5)

#     # Get feature importances specifically for the mRMR selected features
#     importances = rf.feature_importances_

#     # Create a DataFrame for feature importances
#     feature_importances = pd.Series(importances, index=top_features)

#     # Sort by importance score
#     sorted_importances = feature_importances.sort_values(ascending=False)

#     # Create dictionaries for feature rank and importance
#     feature_rank = {feature: rank + 1 for rank, feature in enumerate(sorted_importances.index)}
#     feature_importance = {feature: importance for feature, importance in zip(sorted_importances.index, sorted_importances.values)}

#     # Store results for this K
#     results.append({
#         'K': K,
#         'Accuracy': accuracy_mrmr,
#         'Feature_Rank': feature_rank,
#         'Feature_Importance': feature_importance
#     })

# # Convert results to DataFrame for saving to Excel
# final_results = pd.DataFrame(results)
# # final_results

# file_path_csv = directory + 'mrmr_results.csv'
# final_results.to_csv(file_path_csv, index=False)
# print(f"Results saved to {file_path_csv}")